- k4:
  - score: 0.804
  - macro: 0.575
- k123: 
  - score: 0.669
  - macro: 0.487

In [1]:
import time
import hashlib

import numpy as np
import pandas as pd

In [34]:
from imblearn.over_sampling import SMOTE

from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [39]:
data_x = np.load("data/k_mers/k4_datax.npy")
data_y = np.load("data/k_mers/k4_datay.npy")
print(data_x.shape)
print(data_y.shape)

(2432, 256)
(2432,)


In [40]:
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.20, shuffle=True, random_state=12)

In [42]:
print(np.unique(data_y, return_counts=True))
print(np.unique(y_train, return_counts=True))

(array([0, 1]), array([ 187, 2245]))
(array([0, 1]), array([ 147, 1798]))


In [41]:
clf = svm.SVC(kernel='rbf', C=1, class_weight='balanced').fit(x_train, y_train)
            
score = clf.score(x_test, y_test)
print("score:", score)
y_pred = clf.predict(x_test)
print("macro:", f1_score(y_test, y_pred, average='macro'))

score: 0.8049281314168378
macro: 0.5755699279849549


In [14]:
from scipy import stats

row_count = np.array(row_count)
print(row_count)
z = stats.zscore(row_count, axis=1, ddof=1)
print(row_count.shape)
print(z.shape)

[[7484 5884 4306 ... 6343 5368 5271]
 [2970 2204 1449 ... 2501 3457 2516]
 [2302 2323 1042 ... 1991  876  941]
 ...
 [   3    2    1 ...    5   10    4]
 [   7    2    4 ...    4    2    3]
 [   4    3    9 ...    4    0    1]]
(3998, 16)
(3998, 16)


In [18]:
df = pd.read_csv("data/linear_features/cdhit80_data_seq_loc75_train.csv")

In [20]:
from collections import Counter
def build_vocab(data):
    word_counts = Counter(row.lower() for sample in data for row in sample)
    vocab = [w for w, f in iter(word_counts.items())]
    return vocab
    
def k_mers(data, n):
    kmer_array = [[s[i:i + n].lower() for i in range(len(s) - n)] for s in data]
    vocab = build_vocab(kmer_array)
    return kmer_array, vocab

data_y = np.where(df["loc"].to_numpy() == "Cytosolic", 1, 0)

data_x = None
for k in range(4, 5):
    kmer_array, vocab = k_mers(df["Sequence"], k)
    
    k_array = None
    for x in kmer_array:
        has_v, counts = np.unique(x, return_counts=True)
        
        count_array = []
        for v in vocab:
            if v in has_v:
                count_array.append(counts[list(has_v).index(v)] / sum(counts))
            else:
                count_array.append(0)
        if k_array is None:
            k_array = np.array([count_array])
        else:
            k_array = np.append(k_array, [count_array], axis=0)
        # counter_array = []
        # for v, n in zip(x_counter.keys(), x_counter.values()):
            
        #     print(v)
        #     print(n)
    # print(k_array)
    if data_x is None:
        data_x = k_array
    else:
        data_x = np.append(data_x, k_array, axis=1)
    print(data_x.shape)

(2432, 256)


In [21]:
np.save("data/k_mers/k4_datax", data_x)
np.save("data/k_mers/k4_datay", data_y)